In [ ]:
import requests
import json
import sqlite3
import pandas as pd
import numpy as np
import time

def get_sido_list():
    url = "https://www.starbucks.co.kr/store/getSidoList.do"
    response = requests.post(url)
    data = json.loads(response.text)
    return {item['sido_cd']: item['sido_nm'] for item in data['list'] if item['sido_cd'] and item['sido_nm']}

sido_list = get_sido_list()
sido_list

In [ ]:
def get_stores(sido_cd):
    url = "https://www.starbucks.co.kr/store/getStore.do"
    payload = {
        "r": "PWQMWVRA5Y",
        "in_biz_cds": "0",
        "in_scodes": "0",
        "ins_lat": "37.56682",
        "ins_lng": "126.97865",
        "search_text": "",
        "p_sido_cd": sido_cd,
        "p_gugun_cd": "",
        "isError": "true",
        "in_distance": "0",
        "in_biz_cd": "",
        "iend": "1000",
        "searchType": "C",
        "set_date": "",
        "rndCod": "AKITO9STMQ",
        "all_store": "0"
    }
    response = requests.post(url, data=payload)
    return json.loads(response.text)['list']

sido_cd = '01'

sido_store = get_stores(sido_cd)
df_store = pd.DataFrame(sido_store)
df_store

In [ ]:
df_store = df_store.replace(0, np.nan).replace("", np.nan).dropna(axis=1, how="all")
df_store

In [ ]:
def create_database():
    conn = sqlite3.connect('starbucks_stores.db')
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS stores (
        s_code TEXT PRIMARY KEY,
        s_name TEXT,
        tel TEXT,
        fax TEXT,
        sido_name TEXT,
        gugun_name TEXT,
        addr TEXT,
        doro_address TEXT,
        lat TEXT,
        lot TEXT
    )
    ''')
    conn.commit()
    return conn

def insert_stores(conn, stores):
    cursor = conn.cursor()
    for store in stores:
        cursor.execute('''
        INSERT OR REPLACE INTO stores (s_code, s_name, tel, fax, sido_name, gugun_name, addr, doro_address, lat, lot)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            store['s_code'],
            store['s_name'],
            store['tel'],
            store['fax'],
            store['sido_name'],
            store['gugun_name'],
            store['addr'],
            store['doro_address'],
            store['lat'],
            store['lot']
        ))
    conn.commit()

def main():
    sido_list = get_sido_list()
    conn = create_database()

    for sido_cd, sido_nm in sido_list.items():
        print(f"Collecting stores in {sido_nm}...")
        stores = get_stores(sido_cd)
        print("매장 ",len(stores))
        insert_stores(conn, stores)
        time.sleep(1)  # 서버 부하를 줄이기 위한 대기 시간

    conn.close()

    # 데이터 확인
    conn = sqlite3.connect('starbucks_stores.db')
    df = pd.read_sql_query("SELECT * FROM stores", conn)
    conn.close()

    print(f"\n총 매장 수: {len(df)}")
    print(f"\n지역별 매장 수:\n{df['sido_name'].value_counts()}")

if __name__ == "__main__":
    main()